##Importing Dataset from Kaggle

In [ ]:
from zipfile import ZipFile
file_name = "dataset.zip"
with ZipFile(file_name,'r') as zip:
  zip.extractall()
  print('Done')

BadZipFile: ignored

In [ ]:
!pip install -q kaggle

In [ ]:
! mkdir ~/.kaggle

In [ ]:
cp kaggle.json  ~/.kaggle

In [ ]:
!chmod 600  ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d tolgadincer/labeled-chest-xray-images 

100% 1.17G/1.17G [00:17<00:00, 107MB/s]
100% 1.17G/1.17G [00:17<00:00, 73.7MB/s]


In [ ]:
!unzip labeled-chest-xray-images.zip

##Starting Project

In [ ]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
training_set = train_datagen.flow_from_directory(
        'chest_xray/train',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 5232 images belonging to 2 classes.


In [ ]:
test_datagen = ImageDataGenerator(rescale=1./255)
test_set = test_datagen.flow_from_directory(
        'chest_xray/test',
        target_size=(64, 64),
        batch_size=32,
        class_mode='binary')

Found 624 images belonging to 2 classes.


In [ ]:
cnn=tf.keras.models.Sequential()

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[64,64,3]))

In [ ]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [ ]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

In [ ]:
cnn.add(tf.keras.layers.Flatten())

In [ ]:
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

In [ ]:
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [ ]:
cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
cnn.fit(x=training_set,validation_data=test_set,epochs=25)

Epoch 1/25
164/164 [==============================] - 83s 498ms/step - loss: 0.3824 - accuracy: 0.8347 - val_loss: 0.2527 - val_accuracy: 0.9038
Epoch 2/25
164/164 [==============================] - 80s 488ms/step - loss: 0.2346 - accuracy: 0.9044 - val_loss: 0.3148 - val_accuracy: 0.8702
Epoch 3/25
164/164 [==============================] - 80s 490ms/step - loss: 0.2021 - accuracy: 0.9178 - val_loss: 0.3007 - val_accuracy: 0.8734
Epoch 4/25
164/164 [==============================] - 81s 494ms/step - loss: 0.1751 - accuracy: 0.9304 - val_loss: 0.1692 - val_accuracy: 0.9343
Epoch 5/25
164/164 [==============================] - 80s 489ms/step - loss: 0.1751 - accuracy: 0.9310 - val_loss: 0.2282 - val_accuracy: 0.9119
Epoch 6/25
164/164 [==============================] - 80s 487ms/step - loss: 0.1598 - accuracy: 0.9379 - val_loss: 0.2644 - val_accuracy: 0.8894
Epoch 7/25
164/164 [==============================] - 80s 489ms/step - loss: 0.1539 - accuracy: 0.9360 - val_loss: 0.3711 - val_ac

## Part 4 - Making a single prediction After Some Change More Accurate

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image=image.load_img('dataset/single_prediction/cat_or_dog_1.jpg',target_size=(64,64))
test_image=image.img_to_array(test_image)
######## Add by me ###### feature scaling
test_image=test_image/255.0
#########################
test_image=np.expand_dims(test_image,axis=0)
result=cnn.predict(test_image)
training_set.class_indices
print('result ')
print(result[0][0])
##### given by sir ######
#if result[0][0] ==1:   
#  prediction='dog'
#else:
#  prediction='cat' 
#########################

###### Add by me ####
if result[0][0] >0.5:
  prediction='dog'
else:
  prediction='cat' 
#####################  

In [ ]:
print(prediction)

##Saving our Model

In [ ]:
cnn.save('cat_or_dog_saved_model.h5')

In [ ]:
from keras.models import load_model
new_model=load_model('cat_or_dog_saved_model.h5')

In [ ]:
new_model.summary()

In [ ]:
new_model.get_weights()

In [ ]:
new_model.optimizer

##Prediction through Saved Model

In [ ]:
import numpy as np
from keras.preprocessing import image
test_image=image.load_img('cat_or_dog_15.jpeg',target_size=(64,64))
test_image=image.img_to_array(test_image)
######## Add by me ###### feature scaling
test_image=test_image/255.0
#########################
test_image=np.expand_dims(test_image,axis=0)
result=new_model.predict(test_image)
#training_set.class_indices. ## for determining which index belongs to which class like (Dog =1,Cat =0)
print('result ')
print(result[0][0])
##### given by sir ######
#if result[0][0] ==1:   
#  prediction='dog'
#else:
#  prediction='cat' 
#########################

###### Add by me ####
if result[0][0] >0.5:
  prediction='dog'
else:
  prediction='cat' 
#####################  



In [ ]:
print(prediction)

##Creating TFlite from saved model

In [ ]:
from tensorflow import lite
converter=lite.TFLiteConverter.from_keras_model(new_model)
tfmodel=converter.convert()
open('model.tflite','wb').write(tfmodel)

##Checking our TFlite model

In [ ]:
import numpy as np
import tensorflow as tf
import cv2


from keras.preprocessing import image
img=image.load_img('cat_or_dog_15.jpeg',target_size=(64,64))
img=image.img_to_array(img)
######## Add by me ###### feature scaling
img=img/255.0
#########################
img=np.expand_dims(img,axis=0)

#img = cv2.imread("cat_or_dog_17.jpg")
#img = cv2.resize(img, (64,64))
#img = np.array(img, dtype="float32")
#img = np.reshape(img, (1,64,64,3))


# Load the TFLite model and allocate tensors.
interpreter = tf.lite.Interpreter(model_path="model.tflite")
interpreter.allocate_tensors()

# Get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Test the model on random input data.
input_shape = input_details[0]['shape']

print("*"*50, input_details)
interpreter.set_tensor(input_details[0]['index'], img)

interpreter.invoke()

# The function `get_tensor()` returns a copy of the tensor data.
# Use `tensor()` in order to get a pointer to the tensor.
output_data = interpreter.get_tensor(output_details[0]['index'])
print(output_data)